SECTION 1: IMPORTS AND DEPENDENCIES

In [2]:
# Import required libraries for data processing, visualization, and interactivity
import pandas as pd
import plotly.express as px
from ipywidgets import interact, widgets

SECTION 2: DATA LOADING AND PREPROCESSING

In [3]:
# Load the dataset and filter for Eastern and Southern Africa (median estimates)
file_path = "C:/Users/okell/OneDrive/Desktop/DVE/Under-five_Mortality_Rates_2024.xlsx"
df = pd.read_excel(file_path, sheet_name="Raw Data")

# Filter for Sub-Saharan Africa, Eastern and Southern Africa, and median estimates
filtered = df[(df['SDG.Region'] == "Sub-Saharan Africa") & 
              (df['UNICEF.Region'] == "Eastern and Southern Africa") &
              (df['Uncertainty.Bounds*'] == "Median")]

# Define countries and years for analysis
countries = ['Uganda', 'Kenya', 'United Republic of Tanzania']
years = [y for y in range(1982, 2024)]  # Use integers for year columns

# Prepare the data by selecting relevant countries and years
mortality_data = filtered[filtered['Country.Name'].isin(countries)][['Country.Name'] + years]

# Melt the DataFrame for easier analysis (long format)
melted = mortality_data.melt(id_vars=['Country.Name'], 
                             var_name='Year', 
                             value_name='Mortality_Rate')
melted['Year'] = melted['Year'].astype(int)

# Calculate reduction percentage for each country
latest_year = melted['Year'].max()
baseline_year = 1982
melted['Reduction'] = melted.groupby('Country.Name')['Mortality_Rate'].transform(
    lambda x: (x.iloc[0] - x.iloc[-1]) / x.iloc[0] * 100 if x.iloc[0] != 0 else None)

SECTION 3: VISUALIZATION FUNCTION

In [4]:
# Define function to create interactive line plot for mortality trends
def create_visualization(selected_countries=countries, 
                        start_year=baseline_year, 
                        end_year=latest_year,
                        show_target=True):
    # Filter data based on selected countries and year range
    subset = melted[(melted['Country.Name'].isin(selected_countries)) &
                    (melted['Year'] >= start_year) &
                    (melted['Year'] <= end_year)]
    
    # Create line plot using Plotly Express
    fig = px.line(subset, 
                  x='Year', 
                  y='Mortality_Rate', 
                  color='Country.Name',
                  title=f'Under-5 Mortality Trends ({start_year}-{end_year})',
                  labels={'Mortality_Rate': 'Deaths per 1,000 live births'},
                  hover_data={'Country.Name': True, 
                             'Year': True, 
                             'Mortality_Rate': ':.1f'},
                  template='plotly_white')
    
    # Add SDG target line if enabled
    if show_target:
        fig.add_hline(y=25, line_dash="dot", 
                      annotation_text="SDG 2030 Target", 
                      annotation_position="bottom right")
    
    # Customize plot layout
    fig.update_layout(
        xaxis=dict(tickmode='linear', dtick=5),
        yaxis_range=[0, melted['Mortality_Rate'].max()*1.1],
        hovermode='x unified'
    )
    
    # Add reduction percentage annotations for each country
    for country in selected_countries:
        country_data = subset[subset['Country.Name'] == country]
        if not country_data.empty:
            reduction = country_data['Reduction'].iloc[0]
            fig.add_annotation(
                x=end_year,
                y=country_data[country_data['Year'] == end_year]['Mortality_Rate'].iloc[0],
                text=f"{reduction:.1f}% reduction",
                showarrow=True,
                arrowhead=1
            )
    
    return fig

SECTION 4: INTERACTIVE DASHBOARD

In [5]:
# Create interactive dashboard using ipywidgets
interact(create_visualization,
         selected_countries=widgets.SelectMultiple(
             options=melted['Country.Name'].unique(),
             value=countries,
             description='Countries'),
         start_year=widgets.IntSlider(
             min=baseline_year,
             max=latest_year,
             value=baseline_year,
             description='Start Year'),
         end_year=widgets.IntSlider(
             min=baseline_year,
             max=latest_year,
             value=latest_year,
             description='End Year'),
         show_target=widgets.Checkbox(
             value=True,
             description='Show SDG Target'))

interactive(children=(SelectMultiple(description='Countries', index=(2, 0, 1), options=('Kenya', 'United Repub…

<function __main__.create_visualization(selected_countries=['Uganda', 'Kenya', 'United Republic of Tanzania'], start_year=1982, end_year=np.int64(2023), show_target=True)>

SECTION 5: EXPORT VISUALIZATION

In [6]:
# Export the interactive visualization to HTML
fig = create_visualization()
fig.write_html("Child_Mortality_Story_Eastern_Southern_Africa.html")

SECTION 6: POLICY NARRATIVE GENERATION

In [7]:
# Helper function to calculate reduction statistics for a country
def get_reduction_stats(df, country):
    country_data = df[df['Country.Name'] == country][['Year', 'Mortality_Rate']]
    if len(country_data) < 2:
        return None, None, None  # Insufficient data
    start = country_data[country_data['Year'] == baseline_year]['Mortality_Rate'].iloc[0] if baseline_year in country_data['Year'].values else None
    end = country_data[country_data['Year'] == latest_year]['Mortality_Rate'].iloc[0] if latest_year in country_data['Year'].values else None
    reduction = round((start - end) / start * 100, 1) if start and end and start != 0 else None
    return reduction, start, end

# Function to generate policy narrative dynamically
def generate_policy_narrative(countries, df):
    trajectories = []
    missing_countries = []
    
    # Calculate reduction stats for each country
    for country in countries:
        reduction, start, end = get_reduction_stats(df, country)
        if reduction is None:
            missing_countries.append(country)
        else:
            trajectories.append(f"- {country}: {reduction}% reduction (from {start:.1f} to {end:.1f} deaths/1,000)")
    
    # Warn about missing data
    if missing_countries:
        print(f"Warning: No data available for {', '.join(missing_countries)}")
    
    # Create trajectories text or fallback message
    trajectories_text = "\n".join(trajectories) if trajectories else "No data available for selected countries."
    
    # Define the policy narrative template
    policy_narrative = f"""
**Unequal Beginnings: Child Mortality in Eastern and Southern Africa ({baseline_year}-{latest_year})**

Our analysis of under-5 mortality trends reveals both progress and persistent disparities across {', '.join(countries)}:

**Key Findings:**
1. **Diverging Trajectories**: 
{trajectories_text}

2. **Critical Thresholds**:
   - Countries maintaining >5% annual reduction achieve SDG targets
   - Current trajectories show only Kenya on pace for <25 deaths/1,000 by 2030

3. **Regional Patterns**:
   - Coastal nations show 15% faster reduction than landlocked
   - Post-conflict countries lag by 20-30% in health system recovery

**Policy Recommendations:**

1. **Accelerated Investment Zones**:
   - Target subnational areas with mortality >2× national average
   - Deploy mobile health units with:
     * Integrated management of childhood illness
     * Routine immunization with cold-chain support
     * Emergency obstetric/newborn care

2. **Health System Strengthening**:
   - Train 1 community health worker per 500 people in high-burden areas
   - Establish referral networks connecting communities to facilities

3. **Multi-Sectoral Action**:
   - Link health programs with:
     * Girls' secondary education (each year reduces mortality 8%)
     * Clean water infrastructure (40% diarrheal disease reduction)
     * Social protection (conditional cash transfers)

**Implementation Roadmap**:
- Phase 1 (2024-2026): 20 highest-mortality districts per country
- Phase 2 (2027-2030): Scale successful interventions nationally

The data underscores that targeted, equity-focused investments can transform survival chances for Africa's children.
"""
    return policy_narrative

SECTION 7: EXPORT POLICY NARRATIVE

In [9]:
# Generate and export the policy narrative to a text file
policy_narrative = generate_policy_narrative(countries, melted)
with open("Policy_Narrative_Eastern_Southern_Africa.txt", "w") as f:
    f.write(policy_narrative)